<a href="https://colab.research.google.com/github/fatisepah/samples/blob/CNN-from-scratch/Implement_CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from skimage.util.shape import view_as_windows
from numpy.lib.stride_tricks import as_strided
 
# *******************************************************************************
# Input Layer
# *******************************************************************************
 
input_data_list= [
[-1,1,1, 1, -1],
[-1,1,-1, 1,-1],
[-1,1, 1, 1,-1],
[-1,-1,-1,1,-1],
[-1,-1,-1,1,-1],
[-1,-1,1,-1,-1],
[-1,1,-1,-1,-1]
 
]
 
# Convert list to input data matrix
input_data_matrix = np.asarray(input_data_list, dtype=np.float32)

In [ ]:
# *******************************************************************************
# Convolution layer (convolution operation)
# *******************************************************************************
 
box_filter_list = [
[1, 1,1],
[1,-1,1],
[1, 1,1]
 
]
 
vertical_line_Filter_list = [
[-1, 1, -1],
[-1, 1, -1],
[-1, 1, -1]
 
]
 
Diagonal_line_Filter_list = [
[-1, -1, 1],
[-1, 1, -1],
[1, -1, -1]
 
]
 
# Convert all filter list into matrix
box_filter_matrix = np.asarray(box_filter_list, dtype=np.float32)
vertical_line_Filter_matrix = np.asarray(vertical_line_Filter_list, dtype=np.float32)
Diagonal_line_Filter_matrix = np.asarray(Diagonal_line_Filter_list, dtype=np.float32)
 
# Extract each window from input matrix  by stride operation
def strided4D_v2(input_image_matrix,kernal_matrix,stride):
    return view_as_windows(input_image_matrix, kernal_matrix.shape, step=stride)
 
# Calculate shape of the feature map (output matrix from convolution layer)
featureMap_row = strided4D_v2(input_data_matrix, box_filter_matrix, 1).shape[0]
featureMap_col = strided4D_v2(input_data_matrix, box_filter_matrix, 1).shape[1]
 
# Function to Calculate featuremap matrix for box filter
def conv2d(input_matrix, kernal_matrix):
    # Create blank featureMap matrix for stride 1
    featureMap_Output = np.zeros((featureMap_row, featureMap_col))
 
    for row in range(featureMap_row):
        for col in range(featureMap_col):
            window = strided4D_v2(input_matrix, kernal_matrix, 1)[row][col]
            featureMap_Output[row, col] = np.sum(np.multiply(kernal_matrix, window))
 
            # # To Format floats in a numpy array
            np.set_printoptions(precision=2)
            # Taking average with divided by 9 (total number of element in filter matrix)
            total_number_of_element_in_filter_matrix = kernal_matrix.shape[0] * kernal_matrix.shape[0]
    return (featureMap_Output / total_number_of_element_in_filter_matrix)
 
# ------------------------------------------------------------------------
# Box Filter operation
# ------------------------------------------------------------------------
# Create blank featureMap matrix for stride 1
featureMap_Box = conv2d(input_data_matrix, box_filter_matrix)
print(featureMap_Box)
 
# ------------------------------------------------------------------------
# Vertical line Filter operation
# ------------------------------------------------------------------------
featureMap_Vertical = conv2d(input_data_matrix, vertical_line_Filter_matrix)
# print(featureMap_Vertical)
 
# ------------------------------------------------------------------------
# Diagonal line Filter operation
# ------------------------------------------------------------------------
# Output after applying Diagonal line Filter with stride 1
featureMap_Diagonal = conv2d(input_data_matrix, Diagonal_line_Filter_matrix)
# print(featureMap_Diagonal)

[[-0.11  1.   -0.11]
 [-0.56  0.11 -0.33]
 [-0.33  0.33 -0.33]
 [-0.56 -0.11 -0.56]
 [-0.33 -0.56 -0.33]]


In [ ]:
# *******************************************************************************
# ReLu Operation
# *******************************************************************************
featureMap_Box_ReLu = np.maximum(featureMap_Box, 0)
featureMap_Vertical_ReLu = np.maximum(featureMap_Vertical, 0)
featureMap_Diagonal_ReLu = np.maximum(featureMap_Diagonal, 0)
 
print(featureMap_Box_ReLu)

[[0.   1.   0.  ]
 [0.   0.11 0.  ]
 [0.   0.33 0.  ]
 [0.   0.   0.  ]
 [0.   0.   0.  ]]


In [ ]:
# *******************************************************************************
# Pooling Layer
# *******************************************************************************
 
# Pooling function with stride using python and numpy
def pool2d(input_matrix, kernel_size, stride, padding, pool_mode='max'):
 
    # Padding
    input_matrix = np.pad(input_matrix, padding, mode='constant')
 
    # Window view of input_matrix
    output_shape = ((input_matrix.shape[0] - kernel_size)//stride + 1,
                    (input_matrix.shape[1] - kernel_size)//stride + 1)
    kernel_size = (kernel_size, kernel_size)
    input_matrix_w = as_strided(input_matrix, shape = output_shape + kernel_size,
                        strides = (stride*input_matrix.strides[0],
                                   stride*input_matrix.strides[1]) + input_matrix.strides)
    input_matrix_w = input_matrix_w.reshape(-1, *kernel_size)
 
    # Return the result of pooling
    # For Max Pooling
    if pool_mode == 'max':
        return input_matrix_w.max(axis=(1,2)).reshape(output_shape)
    # For Average Pooling
    elif pool_mode == 'avg':
        return input_matrix_w.mean(axis=(1,2)).reshape(output_shape)
 
# Max Pooling with 2x2 filter & Stride = 1
featureMap_Box_ReLu_MaxPool = pool2d(featureMap_Box_ReLu, kernel_size=2, stride=1, padding=0, pool_mode='max')
featureMap_Vertical_ReLu_MaxPool = pool2d(featureMap_Vertical_ReLu, kernel_size=2, stride=1, padding=0, pool_mode='max')
featureMap_Diagonal_ReLu_MaxPool = pool2d(featureMap_Diagonal_ReLu, kernel_size=2, stride=1, padding=0, pool_mode='max')
print(featureMap_Box_ReLu_MaxPool)

[[1.   1.  ]
 [0.33 0.33]
 [0.33 0.33]
 [0.   0.  ]]


In [ ]:
# *******************************************************************************
# Fully Connected Layer
# *******************************************************************************
# Convert array to list
featureMap_Box_ReLu_MaxPool_list = featureMap_Box_ReLu_MaxPool.tolist()
featureMap_Vertical_ReLu_MaxPool_list = featureMap_Diagonal_ReLu_MaxPool.tolist()
featureMap_Diagonal_ReLu_MaxPool_list = featureMap_Diagonal_ReLu_MaxPool.tolist()
print(featureMap_Box_ReLu_MaxPool_list)
 
# Convert list of list to flat list
featureMap_Box_ReLu_MaxPool_FlatList = [item for sublist in featureMap_Box_ReLu_MaxPool_list for item in sublist]
featureMap_Vertical_ReLu_MaxPool_FlatList = [item for sublist in featureMap_Vertical_ReLu_MaxPool_list for item in sublist]
featureMap_Diagonal_ReLu_MaxPool_FlatList = [item for sublist in featureMap_Diagonal_ReLu_MaxPool_list for item in sublist]
print(featureMap_Box_ReLu_MaxPool_FlatList)
 
# Stack all flat list data
input_to_basic_neural_network = featureMap_Box_ReLu_MaxPool_FlatList + featureMap_Vertical_ReLu_MaxPool_FlatList + featureMap_Diagonal_ReLu_MaxPool_FlatList

[[1.0, 1.0], [0.3333333333333333, 0.3333333333333333], [0.3333333333333333, 0.3333333333333333], [0.0, 0.0]]
[1.0, 1.0, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.3333333333333333, 0.0, 0.0]
